In [20]:
import pandas as pd

In [30]:
df = pd.read_parquet('dfs/player_match_reports.parquet')
df['Date'] = pd.to_datetime(df['Date'], format = "%Y-%m-%d")
df

,Season,League,Team,Date,Player,#,Nation,Pos,Age,Min,...,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog
0,2020-2021,EPL,Fulham,2020-09-12,Aboubakar Kamara,47,mr MTN,FW,25-189,62.0,...,45.0,23.0,0.0,0.0,3.0,4.0,26.0,18.0,69.2,2.0
1,2020-2021,EPL,Fulham,2020-09-12,Aleksandar Mitrović,9,rs SRB,FW,25-362,28.0,...,16.0,3.0,0.0,0.0,1.0,0.0,12.0,7.0,58.3,1.0
2,2020-2021,EPL,Fulham,2020-09-12,Ivan Cavaleiro,17,pt POR,LW,26-330,90.0,...,189.0,76.0,2.0,0.0,2.0,1.0,41.0,34.0,82.9,1.0
3,2020-2021,EPL,Fulham,2020-09-12,Neeskens Kebano,7,cd COD,RW,28-186,62.0,...,134.0,59.0,1.0,0.0,2.0,0.0,26.0,22.0,84.6,3.0
4,2020-2021,EPL,Fulham,2020-09-12,Andre-Frank Zambo Anguissa,29,cm CMR,RW,24-301,28.0,...,124.0,60.0,0.0,0.0,0.0,2.0,20.0,17.0,85.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198136,2017-2018,LaLiga,Real Sociedad,2018-05-20,Raúl Navas,22,es ESP,CB,30-009,90.0,...,144.0,63.0,0.0,0.0,0.0,1.0,35.0,34.0,97.1,2.0
198137,2017-2018,LaLiga,Real Sociedad,2018-05-20,Diego Llorente,3,es ESP,CB,24-277,90.0,...,181.0,128.0,0.0,0.0,1.0,0.0,23.0,23.0,100.0,0.0
198138,2017-2018,LaLiga,Real Sociedad,2018-05-20,Aritz Elustondo,15,es ESP,RB,24-053,71.0,...,75.0,54.0,1.0,0.0,0.0,1.0,19.0,15.0,78.9,1.0
198139,2017-2018,LaLiga,Real Sociedad,2018-05-20,Álvaro Odriozola,19,es ESP,RB,22-157,19.0,...,124.0,89.0,2.0,3.0,0.0,1.0,15.0,13.0,86.7,3.0


In [31]:
columns_to_be_dropped = [
    'Nation',
    '#',
    'Performance_Gls',
    'Performance_Ast',
    'Performance_PK', 
    'Performance_PKatt',
] + [col for col in df.columns if '%' in col ]

df = df.drop(columns_to_be_dropped, 1)

In [32]:
cols_to_be_normalized = list(set(df.columns) - set(['Season',
 'League',
 'Team',
 'Date',
 'Player',
 'Pos',
 'PosNew',
 'Age']))
df[cols_to_be_normalized] = df[cols_to_be_normalized].div(df['Min'], axis=0)

In [33]:
df = df.dropna()

In [34]:
matches_df = pd.read_csv('dfs/matches.csv', parse_dates=['Date'])
matches_df = matches_df.rename(columns={
    'HomeScore': 'Goal_Home', 
    'AwayScore': 'Goal_Away', 
    'HomeTeam': 'Team_Home',
    'AwayTeam': 'Team_Away',
})
matches_df['Win_Home'] = 1 * ((matches_df['Goal_Home'] - matches_df['Goal_Away']) > 0)
matches_df['Win_Away'] = 1 * ((- matches_df['Goal_Home'] + matches_df['Goal_Away']) > 0)
matches_df.head()

,League,Season,Wk,Day,Date,Time,Team_Home,xG_Home,xG_Away,Team_Away,Attendance,Venue,Referee,Match Report,Goal_Home,Goal_Away,Round,Win_Home,Win_Away
0,EPL,2020-2021,1.0,Sat,2020-09-12,12:30,Fulham,0.2,1.8,Arsenal,NaN,Craven Cottage,Chris Kavanagh,https://fbref.com/en/matches/bf52349b/Fulham-A...,0.0,3.0,NaN,0,1
1,EPL,2020-2021,1.0,Sat,2020-09-12,15:00,Crystal Palace,0.7,0.8,Southampton,NaN,Selhurst Park,Jonathan Moss,https://fbref.com/en/matches/db261cb0/Crystal-...,1.0,0.0,NaN,1,0
2,EPL,2020-2021,1.0,Sat,2020-09-12,17:30,Liverpool,3.3,0.6,Leeds United,NaN,Anfield,Michael Oliver,https://fbref.com/en/matches/21b58926/Liverpoo...,4.0,3.0,NaN,1,0
3,EPL,2020-2021,1.0,Sat,2020-09-12,20:00,West Ham,1.1,1.5,Newcastle Utd,NaN,London Stadium,Stuart Attwell,https://fbref.com/en/matches/78495ced/West-Ham...,0.0,2.0,NaN,0,1
4,EPL,2020-2021,1.0,Sun,2020-09-13,14:00,West Brom,0.5,2.2,Leicester City,NaN,The Hawthorns,Anthony Taylor,https://fbref.com/en/matches/7dd01ca9/West-Bro...,0.0,3.0,NaN,0,1


In [35]:
matches_df['game_id'] = matches_df.index

In [36]:
matches_df_long = pd.wide_to_long(matches_df, ["Goal", "xG", 'Team', 'Win'], i="game_id", j="isHome", sep='_', suffix='(!?Home|Away)')
matches_df_long = matches_df_long.drop(['Attendance', 'Round'], 1)
matches_df_long = matches_df_long.reset_index()
matches_df_long


,game_id,isHome,Wk,Day,Date,Match Report,Time,League,Season,Venue,Referee,Goal,xG,Team,Win
0,0,Home,1.0,Sat,2020-09-12,https://fbref.com/en/matches/bf52349b/Fulham-A...,12:30,EPL,2020-2021,Craven Cottage,Chris Kavanagh,0.0,0.2,Fulham,0
1,1,Home,1.0,Sat,2020-09-12,https://fbref.com/en/matches/db261cb0/Crystal-...,15:00,EPL,2020-2021,Selhurst Park,Jonathan Moss,1.0,0.7,Crystal Palace,1
2,2,Home,1.0,Sat,2020-09-12,https://fbref.com/en/matches/21b58926/Liverpoo...,17:30,EPL,2020-2021,Anfield,Michael Oliver,4.0,3.3,Liverpool,1
3,3,Home,1.0,Sat,2020-09-12,https://fbref.com/en/matches/78495ced/West-Ham...,20:00,EPL,2020-2021,London Stadium,Stuart Attwell,0.0,1.1,West Ham,0
4,4,Home,1.0,Sun,2020-09-13,https://fbref.com/en/matches/7dd01ca9/West-Bro...,14:00,EPL,2020-2021,The Hawthorns,Anthony Taylor,0.0,0.5,West Brom,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14623,7309,Away,38.0,Sat,2018-05-19,https://fbref.com/en/matches/207b5248/Villarre...,20:45,LaLiga,2017-2018,Estadio de la Cerámica,José Sánchez,2.0,0.9,Real Madrid,0
14624,7310,Away,38.0,Sun,2018-05-20,https://fbref.com/en/matches/1c4b8a57/Valencia...,12:00,LaLiga,2017-2018,Estadio de Mestalla,David Medié,1.0,1.4,La Coruña,0
14625,7311,Away,38.0,Sun,2018-05-20,https://fbref.com/en/matches/867d08ea/Athletic...,16:15,LaLiga,2017-2018,San Mamés,David Fernández,1.0,1.3,Espanyol,1
14626,7312,Away,38.0,Sun,2018-05-20,https://fbref.com/en/matches/8fbdef5d/Atletico...,18:30,LaLiga,2017-2018,Estadio Wanda Metropolitano,Alfonso Álvarez,2.0,1.0,Eibar,0


In [37]:
matches_df_long[matches_df_long['game_id'] == 0]

,game_id,isHome,Wk,Day,Date,Match Report,Time,League,Season,Venue,Referee,Goal,xG,Team,Win
0,0,Home,1.0,Sat,2020-09-12,https://fbref.com/en/matches/bf52349b/Fulham-A...,12:30,EPL,2020-2021,Craven Cottage,Chris Kavanagh,0.0,0.2,Fulham,0
7314,0,Away,1.0,Sat,2020-09-12,https://fbref.com/en/matches/bf52349b/Fulham-A...,12:30,EPL,2020-2021,Craven Cottage,Chris Kavanagh,3.0,1.8,Arsenal,1


In [38]:
df_grouped = df.groupby(['League', 'Season', 'Team', 'Date']).sum().reset_index()
df_grouped.head()

,League,Season,Team,Date,Min,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,...,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Prog
0,Bundesliga,2017-2018,Augsburg,2017-08-19,13.0,0.212353,0.011111,0.037148,0.0,8.993551,...,0.024098,29.461771,14.487876,0.188034,0.033333,0.372529,0.127912,7.063496,5.618599,1.125885
1,Bundesliga,2017-2018,Augsburg,2017-08-26,14.0,0.578488,0.282999,0.022222,0.0,8.791353,...,0.000000,34.775272,17.064119,0.362155,0.097744,0.184586,0.011111,5.590267,5.493191,1.396032
2,Bundesliga,2017-2018,Augsburg,2017-09-09,14.0,0.134596,0.037374,0.000000,0.0,8.612474,...,0.000000,38.987323,27.332016,0.159916,0.086361,0.201762,0.077778,4.194459,3.421347,0.596896
3,Bundesliga,2017-2018,Augsburg,2017-09-16,14.0,0.255796,0.069191,0.033333,0.0,9.611847,...,0.011236,35.508949,26.155025,0.139327,0.101260,0.187362,0.077903,6.349442,3.861013,0.523460
4,Bundesliga,2017-2018,Augsburg,2017-09-19,14.0,0.202116,0.060317,0.026984,0.0,9.143648,...,0.000000,22.324131,11.109797,0.214815,0.033333,1.193217,0.066916,7.912811,5.902033,1.563183


In [39]:
try_1 = pd.merge(df_grouped, matches_df_long[['League', 'Season', 'Team', 'Date', 'game_id', 'Win', 'isHome', 'Match Report']], how='inner', on=['League', 'Season', 'Team', 'Date'])
try_2 = df_grouped[['League', 'Season', 'Team', 'Date']]
try_3 = matches_df_long[['League', 'Season', 'Team', 'Date', 'game_id', 'Win', 'isHome', 'Match Report']]
try_1.dtypes, try_2.dtypes, try_3.dtypes


(League                    object
 Season                    object
 Team                      object
 Date              datetime64[ns]
 Min                      float64
                        ...      
 Receiving_Prog           float64
 game_id                    int64
 Win                        int64
 isHome                    object
 Match Report              object
 Length: 123, dtype: object,
 League            object
 Season            object
 Team              object
 Date      datetime64[ns]
 dtype: object,
 League                  object
 Season                  object
 Team                    object
 Date            datetime64[ns]
 game_id                  int64
 Win                      int64
 isHome                  object
 Match Report            object
 dtype: object)

In [40]:

len(try_1), len(try_2), len(try_3)

(13958, 13958, 14628)

In [12]:
try_all = try_3.merge(try_2.drop_duplicates(), on=['League', 'Season', 'Team', 'Date'],  how='left', indicator=True)
try_all

,League,Season,Team,Date,game_id,Win,isHome,Match Report,_merge
0,EPL,2020-2021,Fulham,2020-09-12,0,0,Home,https://fbref.com/en/matches/bf52349b/Fulham-A...,both
1,EPL,2020-2021,Crystal Palace,2020-09-12,1,1,Home,https://fbref.com/en/matches/db261cb0/Crystal-...,both
2,EPL,2020-2021,Liverpool,2020-09-12,2,1,Home,https://fbref.com/en/matches/21b58926/Liverpoo...,both
3,EPL,2020-2021,West Ham,2020-09-12,3,0,Home,https://fbref.com/en/matches/78495ced/West-Ham...,both
4,EPL,2020-2021,West Brom,2020-09-13,4,0,Home,https://fbref.com/en/matches/7dd01ca9/West-Bro...,both
...,...,...,...,...,...,...,...,...,...
14623,LaLiga,2017-2018,Real Madrid,2018-05-19,7309,0,Away,https://fbref.com/en/matches/207b5248/Villarre...,both
14624,LaLiga,2017-2018,La Coruña,2018-05-20,7310,0,Away,https://fbref.com/en/matches/1c4b8a57/Valencia...,both
14625,LaLiga,2017-2018,Espanyol,2018-05-20,7311,1,Away,https://fbref.com/en/matches/867d08ea/Athletic...,both
14626,LaLiga,2017-2018,Eibar,2018-05-20,7312,0,Away,https://fbref.com/en/matches/8fbdef5d/Atletico...,both


In [14]:
pd.set_option('display.max_rows', None)
try_all[try_all['_merge'] == 'left_only'].sort_values('Date')

,League,Season,Team,Date,game_id,Win,isHome,Match Report,_merge
11586,Bundesliga,2017-2018,Holstein Kiel,2018-05-17,4272,0,Away,Match Report,left_only
4272,Bundesliga,2017-2018,Wolfsburg,2018-05-17,4272,1,Home,Match Report,left_only
4273,Bundesliga,2017-2018,Holstein Kiel,2018-05-21,4273,0,Home,Match Report,left_only
11587,Bundesliga,2017-2018,Wolfsburg,2018-05-21,4273,1,Away,Match Report,left_only
10356,Ligue1,2017-2018,Toulouse,2018-05-23,3042,1,Away,Match Report,left_only
3042,Ligue1,2017-2018,Ajaccio,2018-05-23,3042,0,Home,Match Report,left_only
10357,Ligue1,2017-2018,Ajaccio,2018-05-27,3043,0,Away,Match Report,left_only
3043,Ligue1,2017-2018,Toulouse,2018-05-27,3043,1,Home,Match Report,left_only
11278,Bundesliga,2018-2019,Union Berlin,2019-05-23,3964,0,Away,Match Report,left_only
3964,Bundesliga,2018-2019,Stuttgart,2019-05-23,3964,0,Home,Match Report,left_only


In [15]:
game_ids = try_all[try_all['_merge'] == 'left_only'].sort_values('Date')['game_id'].values

In [17]:
pd.reset_option('^display.', silent=True)

In [18]:
matches_df[~matches_df['game_id'].isin(game_ids)]

,League,Season,Wk,Day,Date,Time,Team_Home,xG_Home,xG_Away,Team_Away,Attendance,Venue,Referee,Match Report,Goal_Home,Goal_Away,Round,Win_Home,Win_Away,game_id
0,EPL,2020-2021,1.0,Sat,2020-09-12,12:30,Fulham,0.2,1.8,Arsenal,NaN,Craven Cottage,Chris Kavanagh,https://fbref.com/en/matches/bf52349b/Fulham-A...,0.0,3.0,NaN,0,1,0
1,EPL,2020-2021,1.0,Sat,2020-09-12,15:00,Crystal Palace,0.7,0.8,Southampton,NaN,Selhurst Park,Jonathan Moss,https://fbref.com/en/matches/db261cb0/Crystal-...,1.0,0.0,NaN,1,0,1
2,EPL,2020-2021,1.0,Sat,2020-09-12,17:30,Liverpool,3.3,0.6,Leeds United,NaN,Anfield,Michael Oliver,https://fbref.com/en/matches/21b58926/Liverpoo...,4.0,3.0,NaN,1,0,2
3,EPL,2020-2021,1.0,Sat,2020-09-12,20:00,West Ham,1.1,1.5,Newcastle Utd,NaN,London Stadium,Stuart Attwell,https://fbref.com/en/matches/78495ced/West-Ham...,0.0,2.0,NaN,0,1,3
4,EPL,2020-2021,1.0,Sun,2020-09-13,14:00,West Brom,0.5,2.2,Leicester City,NaN,The Hawthorns,Anthony Taylor,https://fbref.com/en/matches/7dd01ca9/West-Bro...,0.0,3.0,NaN,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7309,LaLiga,2017-2018,38.0,Sat,2018-05-19,20:45,Villarreal,1.4,0.9,Real Madrid,18891.0,Estadio de la Cerámica,José Sánchez,https://fbref.com/en/matches/207b5248/Villarre...,2.0,2.0,NaN,0,0,7309
7310,LaLiga,2017-2018,38.0,Sun,2018-05-20,12:00,Valencia,1.5,1.4,La Coruña,35376.0,Estadio de Mestalla,David Medié,https://fbref.com/en/matches/1c4b8a57/Valencia...,2.0,1.0,NaN,1,0,7310
7311,LaLiga,2017-2018,38.0,Sun,2018-05-20,16:15,Athletic Club,1.1,1.3,Espanyol,28492.0,San Mamés,David Fernández,https://fbref.com/en/matches/867d08ea/Athletic...,0.0,1.0,NaN,0,1,7311
7312,LaLiga,2017-2018,38.0,Sun,2018-05-20,18:30,Atlético Madrid,1.2,1.0,Eibar,63229.0,Estadio Wanda Metropolitano,Alfonso Álvarez,https://fbref.com/en/matches/8fbdef5d/Atletico...,2.0,2.0,NaN,0,0,7312
